### California Census Data
### Logistic Regression

In [1]:
import pandas as pd

In [2]:
census = pd.read_csv("census_data.csv")

In [3]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [5]:
census["income_bracket"].unique()

array([' <=50K', ' >50K'], dtype=object)

In [6]:
def label_fix(label):
    if label==' <=50K':
        return 0
    else:
        return 1

In [7]:
census['income_bracket'] = census['income_bracket'].apply(label_fix)

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
x_data = census.drop('income_bracket',axis=1)
y_labels = census['income_bracket']
X_train, X_test, y_train, y_test = train_test_split(x_data,y_labels,test_size=0.3,random_state=101)

In [10]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [13]:
import tensorflow as tf

In [12]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

In [14]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

In [15]:
feat_cols = [gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week]

In [16]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=None,shuffle=True)

In [17]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\jagri\\AppData\\Local\\Temp\\tmpyjlj7log', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002086EA547B8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [18]:
model.train(input_fn=input_func,steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\jagri\AppData\Local\Temp\tmpyjlj7log\model.ckpt.
INFO:tensorflow:loss = 69.31472, step = 1
INFO:tensorflow:global_step/sec: 94.4077
INFO:tensorflow:loss = 350.5074, step = 101 (1.074 sec)
INFO:tensorflow:global_step/sec: 164.823
INFO:tensorflow:loss = 189.57957, step = 201 (0.591 sec)
INFO:tensorflow:global_step/sec: 177.407
INFO:tensorflow:loss = 122.43816, step = 301 (0.564 sec)
INFO:tensorflow:global_step/sec: 172.215
INFO:tensorflow:loss = 108.62299, step = 401 (0.583 sec)
INFO:tensorflow:global_step/sec: 144.175
INFO:tensorflow:loss = 327.91116, step = 501 (0.698 sec)
INFO:tensorflow:global_step/sec: 114.875
INFO:tensorflow:loss = 156.81772, step = 601 (0.875 sec)
INFO:tensorflow:global_step/se

In [19]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [20]:
predictions = list(model.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\jagri\AppData\Local\Temp\tmpyjlj7log\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [21]:
predictions[0]

{'logits': array([-0.59479684], dtype=float32),
 'logistic': array([0.355535], dtype=float32),
 'probabilities': array([0.644465, 0.355535], dtype=float32),
 'class_ids': array([0], dtype=int64),
 'classes': array([b'0'], dtype=object)}

In [22]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [23]:
final_preds[:10]

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

In [24]:
from sklearn.metrics import classification_report

In [25]:
print(classification_report(y_test,final_preds))

             precision    recall  f1-score   support

          0       0.91      0.87      0.89      7436
          1       0.64      0.72      0.68      2333

avg / total       0.84      0.84      0.84      9769

